In [2]:
import pandas as pd
import numpy as np
import re
import os

In [3]:
data_path = '../data/user_detail'
data_list = os.listdir(data_path)
df = pd.DataFrame()
for data in data_list:
    cur_data = pd.read_csv(os.path.join(data_path, data))
    print(len(cur_data), data)
    df = pd.concat([df, cur_data])

10000 user_detail_820000_830000.csv
10000 user_detail_90000_100000.csv
10000 user_detail_270000_280000.csv
10000 user_detail_790000_800000.csv
10000 user_detail_60000_70000.csv
10000 user_detail_30000_40000.csv
10000 user_detail_50000_60000.csv
10000 user_detail_260000_270000.csv
10000 user_detail_550000_560000.csv
10000 user_detail_1_10000.csv
10000 user_detail_160000_170000.csv
10000 user_detail_120000_130000.csv
10000 user_detail_180000_190000.csv
10000 user_detail_70000_80000.csv
10000 user_detail_140000_150000.csv
10000 user_detail_200000_210000.csv
10000 user_detail_20000_30000.csv
10000 user_detail_750000_760000.csv
10000 user_detail_10000_20000.csv
10000 user_detail_130000_140000.csv
10000 user_detail_110000_120000.csv
10000 user_detail_100000_110000.csv
10000 user_detail_210000_220000.csv
10000 user_detail_1_10000_221219.csv
10000 user_detail_740000_750000.csv
10000 user_detail_380000_390000.csv
10000 user_detail_410000_420000.csv
10000 user_detail_190000_200000.csv
10000 user

In [4]:
df = df.reset_index(drop=True)
df_origin = df.copy()

# DATA 기본 전처리
- mureung, theseed, union, achievement : '기록이 없습니다.' 때문에 int로 바꿀 수가 없음
    - NaN을 어떻게 처리할 것인지에 대한 논의 필요 => None으로 하자

### (1) INT로 변경 - level, popularity, total_ranking, world_ranking, class_total_ranking, class_world_ranking

In [5]:
def get_int_basic(data):
    data['level'] = data['level'].apply(lambda x : int(re.search('\d+\(', x).group()[:-1]))
    data['popularity'] = data['popularity'].apply(lambda x : int(x.split('\n')[1].replace(',','')))
    data['total_ranking'] = data['total_ranking'].apply(lambda x : int(x.replace('위', '').replace(',','')))
    data['world_ranking'] = data['world_ranking'].apply(lambda x : int(x.replace('위', '').replace(',','')))   
    data['class_world_ranking'] = data['class_world_ranking'].apply(lambda x : int(x.replace('(월드)','').replace('위', '').replace(',','')))
    data['class_total_ranking'] = data['class_total_ranking'].apply(lambda x : int(x.replace('(전체)','').replace('위', '').replace(',','')))  
    return data 

In [6]:
df_int = get_int_basic(df)

In [7]:
df_int.describe()

,level,popularity,total_ranking,world_ranking,class_world_ranking,class_total_ranking
count,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000
mean,256.917784,397.914375,232475.356137,33535.992409,1427.955959,9759.438328
std,12.252675,2303.375640,239618.423266,38299.907393,2345.695821,14839.181737
min,235.000000,-20533.000000,1.000000,1.000000,1.000000,1.000000
25%,251.000000,7.000000,69986.750000,7721.000000,186.000000,1641.000000
50%,256.000000,46.000000,149818.000000,20118.000000,602.000000,4567.000000
75%,263.000000,202.000000,270064.250000,39146.000000,1626.000000,11353.000000
max,292.000000,99999.000000,830157.000000,161333.000000,19697.000000,102184.000000


### (2) NULL값 변경 -> mureung/theseed/union/achievement
- '기록이 없습니다.'는 None이라는 의미로 -1

In [8]:
def get_int_record(data):
    data['mureung'] = data['mureung'].apply(lambda x : -1 if x == '기록이 없습니다.' else int(x.replace(',','')))
    data['theseed'] = data['theseed'].apply(lambda x : -1 if x == '기록이 없습니다.' else int(x.replace(',','')))
    data['union'] = data['union'].apply(lambda x : -1 if x == '기록이 없습니다.' else int(x.replace(',','')))
    data['achievement'] = data['achievement'].apply(lambda x : -1 if x == '기록이 없습니다.' else int(x.replace(',','')))
    return data

In [9]:
df_rec = get_int_record(df_int)

In [10]:
df_rec.describe()

,level,popularity,total_ranking,world_ranking,class_world_ranking,class_total_ranking,mureung,theseed,union,achievement
count,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000
mean,256.917784,397.914375,232475.356137,33535.992409,1427.955959,9759.438328,35.545756,9.222731,2700.727409,2796.792006
std,12.252675,2303.375640,239618.423266,38299.907393,2345.695821,14839.181737,23.657179,16.798843,3776.888295,4773.502193
min,235.000000,-20533.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,251.000000,7.000000,69986.750000,7721.000000,186.000000,1641.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,256.000000,46.000000,149818.000000,20118.000000,602.000000,4567.000000,46.000000,-1.000000,-1.000000,-1.000000
75%,263.000000,202.000000,270064.250000,39146.000000,1626.000000,11353.000000,51.000000,13.000000,7477.000000,6260.000000
max,292.000000,99999.000000,830157.000000,161333.000000,19697.000000,102184.000000,84.000000,50.000000,10569.000000,26950.000000


### (3) DATE 형태 변경 -> last_access, past_chr_date_1, past_chr_date_2, past_chr_date_3, past_chr_date_4, past_chr_date_5, past_chr_date_6`

In [26]:
def fix_date_format(data:pd.DataFrame) -> pd.DataFrame:
    data['last_access'] = data['last_access'].apply(lambda x : x.replace('/','-'))
    for i in range(1,7):
        data[f'past_chr_date_{i}'] = data[f'past_chr_date_{i}'].apply(lambda x : x.replace('/','-'))
    return data

In [27]:
df_date = fix_date_format(df_rec)